In [ ]:
#!pip install optuna

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os, optuna
import warnings
from PIL import Image
from tensorflow import keras

# from google.colab import drive
# drive.mount("/content/drive")

K = keras.backend
# Changing default dir
os.chdir("/Users/pedroteche/Documents/GitHub/maize-crop-diagnose/")
# optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
# !mkdir 'data'
# !cp -r 'drive/MyDrive/maize-crop-diagnose' 'data'

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

In [ ]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        start_mom=0.95,
        min_mom=0.85,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.start_mom = start_mom
        self.min_mom = min_mom
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
            mom = self._interpolate(
                0, self.half_iteration, self.start_mom, self.min_mom
            )
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
            mom = self._interpolate(
                self.half_iteration,
                2 * self.half_iteration,
                self.min_mom,
                self.start_mom,
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
            mom = self.start_mom
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)
        K.set_value(self.model.optimizer.momentum, mom)


class OneCycleSchedulerNoMom(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [ ]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
EPOCH = 50
# DATA_DIR = "data/maize-crop-diagnose/data/train"
TRAIN_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/train"
TEST_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/test"

In [ ]:

K.clear_session()
train_set, val_set = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
num_classes = len(train_set.class_names)
AUTOTUNE = tf.data.AUTOTUNE
print(train_set.cardinality().numpy())
train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

In [ ]:
TRAIN_SIZE = 11800
TEST_SIZE = 2949

In [ ]:
K.clear_session()
test_set= tf.keras.utils.image_dataset_from_directory(
    TEST_DATA_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
AUTOTUNE = tf.data.AUTOTUNE
test_set = test_set.prefetch(buffer_size=AUTOTUNE).cache()

# Simple input/output example

In [ ]:
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv_in)

In [ ]:
pool_out = keras.layers.GlobalAveragePooling2D()(pool_in)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)

In [ ]:
model = keras.Model(inputs=input, outputs=output)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

# One Residual Unit

In [ ]:
K.clear_session()
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)

In [ ]:
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)

In [ ]:
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r1_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)

model_1r = keras.Model(inputs=input, outputs=output)

In [ ]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=20)

model_1r.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model_1r.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

# 2 Residual Units

In [ ]:
K.clear_session()
# Input layers
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)
# R1
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)
# R2
conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
relu_r2_2 = keras.layers.ReLU()(skip_r2)
# Output layers
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r2_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
# Model
model_2r = keras.Model(inputs=input, outputs=output)

In [ ]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, start_from_epoch=20)

model_2r.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model_2r.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

# 3 Residual Units

In [ ]:
K.clear_session()
# Input layers
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)
# R1
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)
# R2
conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
relu_r2_2 = keras.layers.ReLU()(skip_r2)
# R3
conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding = "same")(relu_r2_2)
conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding = "same")(relu_r2_2)
bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding = "same")(relu_r3_1)
bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
relu_r3_2 = keras.layers.ReLU()(skip_r3)
# Output layers
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r3_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
# Model
model_3r = keras.Model(inputs=input, outputs=output)

In [ ]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, start_from_epoch=20)

model_3r.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model_3r.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)